In [0]:
import requests
import json
import time
import pandas as pd

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.functions import concat


spark= SparkSession \
       .builder \
       .appName("PySpark Server") \
       .getOrCreate()

spark


In [0]:

next = True
page_count=100
payload = {"options":{"cellFormat":"string"},"pageContext":None,"pagingOption":{"offset":None,"count":page_count},"filterCriteria":{}} 
columns = ['id', 'Ethnicity', 'Age','Gender', 'Appearance' ]
limit =500
count=0
while next and count<=limit:
    data=[]
    r=requests.post(url="""https://profiles.sunnahnikah.co/v1/datasource/airtable/53c28e05-67eb-4322-be23-ea1ac768c030/5a6b22a4-3c1c-411a-b1f0-2671ada07826/7f5c2698-daa0-408d-856d-f4a773818f77/eee4fb36-5b78-4c00-81d3-2f59d5b5ade6/data""", json=payload).json()
    for i in range(len(r.get("records"))):
        data.append((r.get("records")[i]["id"],r.get("records")[i].get("fields").get("Ethnicity"),r.get("records")[i].get("fields").get("Age"),r.get("records")[i].get("fields").get("Gender"), r.get("records")[i].get("fields").get('Appearance')))
        
    if count==0:
        #init df
        profiles_df = spark.createDataFrame(data = data, schema= columns)
    else:
        temp_df = spark.createDataFrame(data = data, schema= columns)
        profiles_df=profiles_df.union(temp_df)

    if r.get("offset") is None:
        next=False
    payload["pagingOption"]["offset"] = r["offset"]
    #print(data)
    time.sleep(2.5)
    count+=1
    print(count)
    print(r.get("records")[i].get("fields"))




In [0]:
profiles_df.show(100)

In [0]:
profiles_pandas_df = profiles_df.toPandas()
def clean(x):
    if x[2] =='':
        x[2] = 999
    x[4] = x[4].strip()

    if x[4] == '':
        x[4] == 'Nan'
    elif x[4][-1]==',':
        x[4] = x[4][:-1]
    return [x[0], x[1], int(x[2]), x[3], x[4]]
profiles_pandas_df=profiles_pandas_df.apply(clean, axis=1, result_type='broadcast')
profiles_pandas_df['Age'] = profiles_pandas_df['Age'].astype(int)

In [0]:
print(profiles_pandas_df)

In [0]:
# create clean spark dataframe
profiles_clean_df  = spark.createDataFrame(data=profiles_pandas_df)

# write it to csv
profiles_clean_df.write.mode("overwrite").format("csv").option("header", "true").save("/Volumes/sn_demo/default/vol1/sn_profiles")

# save to sql warehouse
catalog = "dev"
schema = "default"
table_name = "sn_profiles"
path_table = catalog + "." + schema
print(path_table) # Show the complete path
profiles_clean_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{path_table}.{table_name}")
profiles_clean_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{path_table}.{table_name}")


In [0]:
def perform_hist_analysis(df:pd.DataFrame):

    hist=df.hist('Age', bins=50)
    print(F"mean age: {df['Age'].mean()}")
    print(F"most frequent age: {df['Age'].mode()}")
    hist[0][0].set_xlim(0,100)
    return None

In [0]:
perform_hist_analysis(profiles_pandas_df[profiles_pandas_df["Gender"]=="Male"])
net=profiles_pandas_df.shape[0]
print(f"Fraction of Males: {profiles_pandas_df[profiles_pandas_df['Gender']=='Male'].shape[0]/net}")

In [0]:
perform_hist_analysis(profiles_pandas_df[profiles_pandas_df["Gender"]=="Female"])
print(f"Percentage of Females:{profiles_pandas_df[profiles_pandas_df['Gender']=='Female'].shape[0]/net}")

In [0]:
profiles_pandas_df.Appearance.unique()

In [0]:
apperance_count =profiles_pandas_df.groupby('Appearance').count()
print(apperance_count)
hist=apperance_count.plot.bar(y='id')